In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt
import seaborn as sns

In [70]:
test_data = pd.read_excel(r"Test_set.xlsx")

In [71]:
test_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [72]:
test_data['Destination'].unique()

array(['Cochin', 'Banglore', 'Delhi', 'New Delhi', 'Hyderabad', 'Kolkata'],
      dtype=object)

In [73]:
test_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [74]:
# Extracting day and month from Date of Journey
test_data['journey_day'] = pd.to_datetime(test_data.Date_of_Journey , format = "%d/%m/%Y").dt.day
test_data['journey_month'] = pd.to_datetime(test_data.Date_of_Journey , format = "%d/%m/%Y").dt.month
test_data.drop(["Date_of_Journey"], axis = 1, inplace = True)

In [75]:
## Extracting hour and minute from departure time
test_data['Dep_hour'] = pd.to_datetime(test_data.Dep_Time).dt.hour
test_data['dep_min'] = pd.to_datetime(test_data.Dep_Time).dt.minute
test_data.drop(["Dep_Time"], axis = 1, inplace = True)

In [76]:
## exatrcting hour and minute from arrival time
test_data['Arrival_hour']= pd.to_datetime(test_data.Arrival_Time).dt.hour
test_data['Arrival_minute']= pd.to_datetime(test_data.Arrival_Time).dt.minute
test_data.drop(["Arrival_Time"], axis =1, inplace = True)


In [77]:
## Time taken for flight to reacg destination
duration = list(test_data['Duration'])
for i in range(len(duration)):
    if len(duration[i].split())!= 2: 
        
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
        else:
            duration[i] = "0h " + duration[i]           # Adds 0 hour
            duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))   
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))
test_data['duration_hours']= duration_hours
test_data['duration_mins'] = duration_mins
test_data.drop(["Duration"], axis = 1, inplace = True)

In [78]:
test_data.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,journey_day,journey_month,Dep_hour,dep_min,Arrival_hour,Arrival_minute,duration_hours,duration_mins
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,6,6,17,30,4,25,10,55
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,1 stop,No info,12,5,6,20,10,20,4,0
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,In-flight meal not included,21,5,19,15,19,0,23,45
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,21,5,8,0,21,0,13,0
4,Air Asia,Banglore,Delhi,BLR → DEL,non-stop,No info,24,6,23,55,2,45,2,50


# Feature Enginnering
## Handling categorical variables

In [81]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [82]:
for i in test_data.columns:
    if test_data[i].dtypes=='object':
        test_data[i] = encoder.fit_transform(test_data[i])

In [85]:
## Normalising the data
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled_data=scaler.fit_transform(test_data)

In [87]:
scaled_data= pd.DataFrame(data=scaled_data, columns=test_data.columns)
scaled_data.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,journey_day,journey_month,Dep_hour,dep_min,Arrival_hour,Arrival_minute,duration_hours,duration_mins
0,0.4,0.50,0.2,0.767677,0.0,1.0,0.192308,1.000000,0.739130,0.545455,0.173913,0.454545,0.250,1.000000
1,0.3,0.75,0.0,0.656566,0.0,1.0,0.423077,0.666667,0.260870,0.363636,0.434783,0.363636,0.100,0.000000
2,0.4,0.50,0.2,0.767677,0.0,0.6,0.769231,0.666667,0.826087,0.272727,0.826087,0.000000,0.575,0.818182
3,0.6,0.50,0.2,0.767677,0.0,1.0,0.769231,0.666667,0.347826,0.000000,0.913043,0.000000,0.325,0.000000
4,0.0,0.00,0.4,0.161616,1.0,1.0,0.884615,1.000000,1.000000,1.000000,0.086957,0.818182,0.050,0.909091


In [88]:
scaled_data.to_csv(r'test.csv')